# Statistical Inference of Luminosity and Temperature of Dwarf Stars

### Ahmed Rizk, Cathy Liu, Divya Bilolikar, Olivia Pang

In [ ]:
library(tidyverse)
library(infer)
library(gridExtra)
library(testthat)
library(digest)
library(broom)

## Introduction

Stars are typically classified into four different groups – white dwarfs, main sequence, giants, and supergiants (Chiosi, 1992). A typical star will become a main sequence star, then end its life cycle as a white dwarf. This is the most frequent lifecycle that we see (Tillman, 2022). White dwarfs are very small dense stars that are not very bright when compared to main-sequence stars (Koester, 1990). Contrary to their name, the red dwarfs are not a type of white dwarf but rather a type of small main-sequence star that is not very bright. Similarly, we have brown dwarfs, which are not usually classified as stars because they do not burn hot enough. (Tillman, 2019) When a massive star is formed, it will become a giant star or a supergiant and then end its lifecycle as a neutron star or a blackhole. Giants and supergiants are enormous stars that shine very bright when compared to main-sequence stars (Chiosi, 1992). 

While white, red, and brown dwarf stars share a name, they are very different and typically not classified together. This is clearly seen in an Hertzsprung-Russell diagram (a scatter plot that shows the trends in star types against luminosity, temperature, color, and magnitude) where white dwarfs have their own classification in the bottom left, and red and white dwarfs are grouped together in the main sequence group towards the bottom right. Since all three stars appear on the bottom half of the diagram we want to explore the difference in luminosity and temperature between the three.


## Analysis Question
In this project, we want to determine if there is a difference in luminosity and temperature between red, brown, and white dwarf stars. Our location parameter will be the mean of luminosity and temperature and our scale parameter will be the standard deviation of both luminosity and temperature. We chose these because we have a large dataset (240 observations) and thus the CLT takes effect, and we can treat the dataset as normally distributed.


We are using the "Star dataset to predict star types" dataset from https://www.kaggle.com/datasets/deepu1109/star-dataset. The variables are as follows:

| Variable | Description |
| :-: | :-: |
| Absolute Temperature (K) | The temperature of the star in Kelvin |
| Relative Luminosity (L/Lo) | The luminosity of the star when divided by the average luminosity of the sun (3.828 x 10^26 Watts) |
| Relative Radius (R/Ro) | The radius of the star when divided by the average radius of the sun (6.9551 x 10^8 m) |
| Absolute Magnitude (Mv) | The absolute magnitude of the star |
| Star Color | The color of the star |
| Spectral Class (O,B,A,F,G,K,M) | If the star is a main sequence star, the type of main sequence star it is |
| Star Type (Red Dwarf, Brown Dwarf, White Dwarf, Main Sequence , SuperGiants, HyperGiants) | The type of star it is | 

## Preliminary Analysis

### Reading the dataset into R

In [ ]:
stars <- read.csv("6-class.csv")

glimpse(stars)

### Cleaning and Wrangling

Since the target categorical variable for our analysis is the star type, we need to change it from type 'int' to type 'factor'.

In [ ]:
stars <- mutate(stars, Star.type = as_factor(Star.type))

Next, we filter out the Giant stars (types 3, 4 and 5) to isolate the Dwarfs.

In [ ]:
dwarfs <- stars %>%
filter(Star.type != 5) %>%
filter(Star.type != 4) %>%
filter(Star.type != 3)


### Visualizations

We can create boxplots to show the variation in luminosity and temperature across star types. This will give us a rough idea of the differences between each star type, and tell us if any groups are vastly different from the rest.

In [ ]:
L_box <- ggplot(dwarfs, aes(x = Star.type, y = Luminosity.L.Lo.)) + geom_boxplot() + ggtitle("Luminosity vs Star Type") + xlab("Star Type") + ylab("Luminosity (Lo)")
T_box <- ggplot(dwarfs, aes(x = Star.type, y = Temperature..K.)) + geom_boxplot() +  ggtitle("Temperature vs Star Type") +  xlab("Star Type") + ylab("Temperature (K)")
grid.arrange(L_box, T_box, ncol=2)

In the figure above we can see the distributions of Luminosity vs Star type and Temperature vs Star type. The means of the luminosity across star types are very similar, however, the interquartile range of star type 1 is much larger. In the Temperature vs Star type graph we can see that the range and mean of star type 0 and type 1 are very similar, however, both the mean and interquartile range of star type 2 is very large.


To find the within group distributions for each star type, we will separate the data into 3 different parts, each representing a star type.

In [ ]:
type_0 <- filter(stars, Star.type == 0)
type_1 <- filter(stars, Star.type == 1)
type_2 <- filter(stars, Star.type == 2)

#### Brown Dwarf Distributions

In [ ]:
# Find the distribution of luminosity and temperature values for the brown dwarf star

L_0 <- ggplot(type_0, aes(x = Luminosity.L.Lo.)) + geom_histogram(bins = 20)
T_0 <- ggplot(type_0, aes(x = Temperature..K.)) + geom_histogram(bins = 20)
grid.arrange(L_0, T_0, ncol=2)

In the Luminosity graph above, we can see that the distribution is unimodal and looks fairly normal with a few larger outliers. Most stars seem to have a luminosity between 0 and 0.001 LL_o. In the Temperature graph, we can see that the distribution looks bimodal peaking at around 2800 K and 3500 K.

#### Red Dwarf Distributions

In [ ]:
# Find the distribution of luminosity and temperature values for the red dwarf star

L_1 <- ggplot(type_1, aes(x = Luminosity.L.Lo.)) + geom_histogram(bins = 20)
T_1 <- ggplot(type_1, aes(x = Temperature..K.)) + geom_histogram(bins = 20)
grid.arrange(L_1, T_1, ncol=2)

Similar to the brown dwarfs, the distribution of the Luminosity graph is unimodal and looks fairly normal with a few larger outlier. Most stars seem to have a luminosity between 0 and 0.01 LL_o. In the Temperature graph we can see that the distribution looks unimodal peaking in the range of 3100K to 3500K.

#### White Dwarf Distributions

In [ ]:
# Find the distribution of luminosity and temperature values for the white dwarf star

L_2 <- ggplot(type_2, aes(x = Luminosity.L.Lo.)) + geom_histogram(bins = 20)
T_2 <- ggplot(type_2, aes(x = Temperature..K.)) + geom_histogram(bins = 20)
grid.arrange(L_2, T_2, ncol=2)

Finally, the Luminosity graph for white dwarfs has the same characteristics as the other two. Most stars seem to have a luminosity between 0 and 0.005 LL_o. In the Temperature graph we can see that the distribution looks right skewed and unimodal.

### Computing Parameter Estimates

#### Brown Star Estimates

In [ ]:
# Finding the mean and standard deviation of the luminosity and temperature of the brown star

brown_dwarf_stats <- type_0 %>% 
    summarise(mean_lum = mean(Luminosity.L.Lo.), 
              sd_lum = sd(Luminosity.L.Lo.),
              mean_temp = mean(Temperature..K.),
              sd_temp = sd(Temperature..K.)) %>%
    add_column(Type = "Brown", .before = "mean_lum")

brown_dwarf_stats

In [ ]:
# Finding the mean and standard deviation of the luminosity and temperature of the red star

red_dwarf_stats <- type_1 %>% 
    summarise(mean_lum = mean(Luminosity.L.Lo.), 
              sd_lum = sd(Luminosity.L.Lo.),
              mean_temp = mean(Temperature..K.),
              sd_temp = sd(Temperature..K.)) %>%
    add_column(Type = "Red", .before = "mean_lum")

red_dwarf_stats

In [ ]:
# Finding the mean and standard deviation of the luminosity and temperature of the white star

white_dwarf_stats <- type_2 %>% 
    summarise(mean_lum = mean(Luminosity.L.Lo.), 
              sd_lum = sd(Luminosity.L.Lo.),
              mean_temp = mean(Temperature..K.),
              sd_temp = sd(Temperature..K.)) %>%
    add_column(Type = "White", .before = "mean_lum")

white_dwarf_stats

In [ ]:
# Combining all parameter estimates together

parameter_estimates <- rbind(brown_dwarf_stats, red_dwarf_stats, white_dwarf_stats)
parameter_estimates

## Methods

This report is trustworthy because the dataset that we use is fairly large, so it is potentially representative of the true population. Therefore the mean estimates that we have calculated can be good starting points to create confidence intervals for our population parameters. While our estimations are a good starting point, they are not enough to present to a stakeholder. The plots would be a better representation if we were to do some further data processing like bootstrapping or asymptotic testing. Additionally there is no way to estimate the sampling variability of the population of stars by examining point estimates of the standard deviation, or the distribution of values of the dataset. 

In order to address the gap we brought up above, we need to bootstrap to more closely approximate the actual distribution of the population, and by extension find a range of possible parameter estimates as opposed to a point estimate.
In the final report, we expect to find whether we are confident to say there is a difference in the luminosity and temperature between dwarf stars. We will express our result and the uncertainty in a trustworthy way. We believe finding this clarifies the difference in some features between different types of dwarf stars, and further increases the accuracy and reliability of star classification. This analysis leads to future questions such as “is there a difference in the magnitude between dwarf stars”. Additionally we can use these results to infer real astronomical observations.

Since our analysis will examine the means of 2 variables for 3 different groups, using 2 hypotheses would minimize the probability of making a type 1 error from the sample. We can carry this out by using an ANOVA test twice, one for the means of each variable.

# Bootstrap Analysis

In the following code cells we conduct a bootstrap on each of the different types of dwarf stars in order to find better estimates for the means and standard deviations of both luminosity and temperature. 

The first step in doing a bootstrap distribution is creating a bootstrap distribution of size 40 with 1000 replicates. Then we find the mean and standard deviation of each sample. We do this because we want to approximate the variance of the bootstrap distribution. Then we found the mean and standard deviation of the whole distribution. We do this for each type of star we have.

### Bootstrap on Temperature and Luminosity of Brown Dwarfs

In [ ]:
brown_dwarf_stats <- type_0 %>% 
    rep_sample_n(size = 40, reps = 1000, replace = TRUE) %>% #Creating the bootstrap distribution. 
    group_by(replicate) %>% 
    summarise(mean_lum = mean(Luminosity.L.Lo.), #Finding means and standard deviations of each sample. 
              mean_temp = mean(Temperature..K.),
              sd_lum = sd(Luminosity.L.Lo.),
              sd_temp = sd(Temperature..K.)) %>%
    summarise(mean_lum = mean(mean_lum), #Using means to summarizing the estimates. 
              mean_temp = mean(mean_temp), 
              SE_lum = mean(sd_lum), 
              SE_temp = mean(sd_temp)) %>%
    add_column(Type = "Brown", .before = "mean_lum")
brown_dwarf_stats

We can see that for brown dwarfs bootstrap distribution our mean luminosity is 6.92e-4 L/Lo and its associated standard error is 7.95e-4 L/Lo. For our temperature bootstrap distribution the mean is 2996.97 K and its associated standard error is 325.7 K.

### Bootstrap on Temperature and Luminosity of Red Dwarfs

In [ ]:
red_dwarf_stats <- type_1 %>% 
    rep_sample_n(size = 40, reps = 1000, replace = TRUE) %>% 
    group_by(replicate) %>% 
    summarise(mean_lum = mean(Luminosity.L.Lo.), 
              mean_temp = mean(Temperature..K.),
              sd_lum = sd(Luminosity.L.Lo.),
              sd_temp = sd(Temperature..K.)) %>%
    summarise(mean_lum = mean(mean_lum), 
              mean_temp = mean(mean_temp), 
              SE_lum = mean(sd_lum), 
              SE_temp = mean(sd_temp)) %>%
    add_column(Type = "Red", .before = "mean_lum")
red_dwarf_stats

We can see that for red dwarfs bootstrap distribution our mean luminosity is 5.45e-3 L/Lo and its associated standard error is 7.03e-3 L/Lo. For our temperature bootstrap distribution the mean is 3284.7 K and its associated standard error is 265.2 K.

### Bootstrap on Temperature and Luminosity of White Dwarfs

In [ ]:
white_dwarf_stats <- type_2 %>% 
    rep_sample_n(size = 40, reps = 1000, replace = TRUE) %>% 
    group_by(replicate) %>% 
    summarise(mean_lum = mean(Luminosity.L.Lo.), 
              mean_temp = mean(Temperature..K.),
              sd_lum = sd(Luminosity.L.Lo.),
              sd_temp = sd(Temperature..K.)) %>%
    summarise(mean_lum = mean(mean_lum), 
              mean_temp = mean(mean_temp), 
              SE_lum = mean(sd_lum), 
              SE_temp = mean(sd_temp)) %>%
    add_column(Type = "White", .before = "mean_lum")
white_dwarf_stats

We can see that for brown dwarfs bootstrap distribution our mean luminosity is 2.46e-3 L/Lo and its associated standard error is 7.47e-3 L/Lo. For our temperature bootstrap distribution the mean is 13937.92 K and its associated standard error is 4884.55 K.

In [ ]:
bootstrap_estimates <- rbind(brown_dwarf_stats, red_dwarf_stats, white_dwarf_stats)
bootstrap_estimates

Above is a summary of all the statistics of the bootstrap for each star type. We can see that the mean luminosity of brown dwarfs is significantly smaller (a power of 10) than the mean luminosity of red and white dwarfs. Similarly the standard error of brown dwarfs is significantly smaller (by a power of 10) than the standard error of red and white dwarfs. 

The mean temperature of white dwarfs is larger than the means of red and brown dwarfs by a power of 10. The standard error of white dwarfs is larger than the standard error of the other two by a power of 10.

# ANOVA

ANOVA (Analysis of Variance) is a test to determine whether the difference between means across three or more groups exist by analyzing the overall and within-group variance. ANOVA test calculates F-value, which is the ratio of the variability between group means and the variation within groups, and corresponding p-value. If F-value is significantly greater than 1, the p-value is going to be small, we conclude that the variability between group means exists. In this project, since our goal is to analyze the difference between temperature and luminosity of three groups of dwarf planets, ANOVA becomes very appropriate and effective. In this section, we will conduct two ANOVA tests across three dwarf groups on both variables, and analyze the results.


### ANOVA for Temperature Across all Three Dwarf Types

We start with temperature. Before actually starting the ANOVA test, we need to do a little bit of cleanup, select the columns that we want, which are Star Type and Temperature.

In [ ]:
# filter out the target column Temperature to get the tidy data set
dwarfs_temp <- dwarfs %>%
               select(Temperature..K., Star.type)
head(dwarfs_temp)

The resulting table contains two columns, temperature and star type. Temperature is a numeric variable which is the target of the test. Star type is a factor indicating groups for ANOVA.
In the following two blocks of code, we perform ANOVA test on temperature across three groups of dwarfs.

In [ ]:
# Perform ANOVA test and print the result table
anova_temp <- aov(Temperature..K.~Star.type, data = dwarfs_temp)%>%
              tidy()
print(anova_temp)

In [ ]:
# Filter out the F-value from the table above
# Filter out the p-value from the table above
f_stat_temp <- anova_temp %>%
          filter(term == "Star.type") %>%
          select(statistic) %>%
          as.numeric()
anova_pval_temp <- anova_temp %>%
              filter(term == "Star.type") %>%
              select(p.value) %>%
              as.numeric()
print(f_stat_temp)
print(anova_pval_temp)

After performing ANOVA test on temperature, we obtained the F-value = 188.91, and the p-value = 2.74e-37. 

### ANOVA for Luminosity Across all Three Dwarf Types

In this piece of code, we cleanup the original data set by selecting luminosity and star type columns in order to prepare for the ANOVA test.

In [ ]:
# filter out the target column Luminosity to get the tidy data set
dwarfs_lumi <- dwarfs %>%
               select(Luminosity.L.Lo., Star.type)
head(dwarfs_lumi)

The resulting table contains two columns, luminosity and star type. In this case, luminosity is a numeric variable served as the target of the test. Star type is an explanatory factor indicating groups for ANOVA. In the following two blocks of code, we perform ANOVA test on luminosity across three groups of dwarfs.

In [ ]:
# Perform ANOVA test and print the result table
anova_lumi <- aov(Luminosity.L.Lo.~Star.type, data = dwarfs_lumi)%>%
              tidy()
print(anova_lumi)

In [ ]:
# Filter out the F-value from the table above
# Filter out the p-value from the table above
f_stat_lumi <- anova_lumi %>%
          filter(term == "Star.type") %>%
          select(statistic) %>%
          as.numeric()
anova_pval_lumi <- anova_lumi%>%
              filter(term == "Star.type") %>%
              select(p.value) %>%
              as.numeric()
print(f_stat_lumi)
print(anova_pval_lumi)

After performing ANOVA test on luminosity, we obtained the F-value = 5.09, and the p-value = 0.0076. 

# Results

The table below summarizes the result from two ANOVA tests.

In [ ]:
# Result table to show the obtained F-values and P-values
table <- data.frame (Type  = c("F-Value", "P-Value"),
                     Temperature_Result = c(f_stat_temp, anova_pval_temp),
                     Luminosity_Result = c(f_stat_lumi, anova_pval_lumi))
table

In the temperature ANOVA test, the F-value obtained is 188.91, and the p-value is 2.74e-37. In this case, our F-value is significantly greater than 1. Therefore, the variation between group means is clearly larger than the variation within groups, indicating that at least one of the mean value is different across three groups. The p-value also states the result. Since p-value is significantly smaller than the critical value (0.05), it falls in reject region. Therefore, by looking at the ANOVA test for temperature across three groups, we say at least one of the mean value is different across three groups and the temperature of three dwarf groups are not all same.

In the luminosity ANOVA test, The F-value obtained is 5.09, and the p-value is 0.0076. Although we get a much smaller F-value than the F-value in temperature test, it still clearly larger than 1. The p-value is also smaller than the critical value (0.05), and falls in reject region. Although we might say the difference between luminosity across groups is not as significantly as the difference between temperature, we are still confident to say at least one mean value is different from others, and the luminosity of three dwarf groups are not all same.

# Discussion

The results of our temperature ANOVA test showed that we got a p-value of 2.74e-37. This is significantly less than our alpha level of 0.05. As a result we reject the Null Hypothesis and conclude that there is a difference in the mean temperature of stars across the three categories (brown dwarfs, red dwarfs, and white dwarfs). From this one way ANOVA test we cannot conclude how the means differ just that all three means are not the same.
 
The results of our luminosity ANOVA test showed that we got a p-value of 0.008. One again this is less than our alpha level of 0.05 and thus we reject the Null Hypothesis that the means of the luminosity are the same across all three star types. 

We expected these results because from our exploratory data analysis we could visually see that the means of the star types for both temperature and luminosity were very different. 

Even though the probability of a type I error is very small, we do not believe that we are at risk of committing a type II error. As the probability of type I error decreases, the probability of committing a type II error increases however we feel like that is offset by our large sample size.

## Implication of Findings

The results of our ANOVA test indicate that there do exist differences between the three types of dwarfs in terms of temperature and luminosity. We are now relatively confident that the three types of dwarfs do not have the same average temperature and luminosity. 
First of all, we believe this finding further clarifies the difference between dwarf stars, giving a better understanding of stars and celestial bodies in the universe. 

In the article Red dwarfs: The most common and longest-lived stars, the author states that there is no true definition of red dwarf; the classification just refers to dwarf stars with different spectral types (Tillman, 2019). Therefore, we hope our finding helps to improve the accuracy of the star classification system. We also believe that the results could be easily applied to the real-world situation when people are doing actual astronomical analysis. For example, if anyone tries to clarify between multiple groups of dwarf stars, the results of this project could help them make a meaningful guess based on temperature and luminosity data. 

Furthermore, the difference in temperature and luminosity could indicate the existence of other differences which have not yet been found by people. It leads to various research topics on the reasons that cause these differences and impact of these differences on our current knowledge of celestial bodies.

## Limitations of Analysis

There are several limitations of our analysis. The first limitation is that although using ANOVA would be far better at limiting the probability of type 1 error, our groups do not have roughly equal variances. The temperature of white stars has a variance of $4957.6552^2$ which far exceeds that of the other 2 groups. A similar phenomenon is observed in the luminosity of brown stars, which is far less than the other 2 types. Therefore, our F-scores for both variables are somewhat inflated. 

A second limitation is that although our dataset is large enough (240 observations), our within-group distributions for the luminosity of white/brown/red stars are right skewed. Therefore, we might need a larger sample for the CLT to take effect for luminosity.


## Further Questions

Since our EDA points to which type of star varies in terms of the means and variances from the rest, a possible future analysis could be done to explore the temperatures and luminosities of the (red, brown) and the (red, white) stars respectively. 

Additionally we can also explore other star variables such as magnitude and color and see how their means differentiate.


# References

Chiosi, Cesare, Gianpaolo Bertelli, and Alessandro Bressan. "New developments in understanding the HR diagram." Annual review of astronomy and astrophysics 30.1 (1992): 235-285.

Koester, Detlev, and Ganesar Chanmugam. "Physics of white dwarf stars." Reports on Progress in Physics 53.7 (1990): 837.

Tillman, Nola Taylor, and Ben Biggs. “Main Sequence Stars: Definition & Life Cycle.” Space.com, Space, 26 Jan. 2022, https://www.space.com/22437-main-sequence-star.html. 

Tillman, Nola Taylor. “Red Dwarfs: The Most Common and Longest-Lived Stars.” Space.com, Space, 6 June 2019, https://www.space.com/23772-red-dwarf-stars.html. 